In [2]:
from casacore.tables import table
import numpy as np

import idg.util as util
import idg

Error importing OpenCL: ('/opt/lib/libidg-opencl.so: cannot open shared object file: No such file or directory',)


Notes on IO: It's quite a bit faster to read everything and then zero out the autocorr than use `tcross  = t.query('ANTENNA1!=ANTENNA2')` to get the cross corrs.

On 1chan 9000 int, the read with query takes 6 min and read with zeroing out autocorrs takes 4min.

In [3]:
with table('/fastpool/data/20210226M-2GHz-1chan-600int.ms') as t:
    t_ant = table(t.getkeyword("ANTENNA"))
    t_spw = table(t.getkeyword("SPECTRAL_WINDOW"))
    frequencies = np.asarray(t_spw[0]['CHAN_FREQ'], dtype=np.float32)

Successful readonly open of default-locked table /fastpool/data/20210226M-2GHz-1chan-600int.ms: 25 columns, 1257676800 rows
Successful readonly open of default-locked table /fastpool/data/20210226M-2GHz-1chan-600int.ms/ANTENNA: 8 columns, 2048 rows
Successful readonly open of default-locked table /fastpool/data/20210226M-2GHz-1chan-600int.ms/SPECTRAL_WINDOW: 14 columns, 1 rows


In [2]:
%%time
with table('/fastpool/data/20210226M-2GHz-1chan-600int.ms') as t:
    tcross  = t.query('ANTENNA1!=ANTENNA2')
    vis = tcross.getcol('DATA')
    ant1 = tcross.getcol('ANTENNA1')
    ant2 = tcross.getcol('ANTENNA2')
    uvw = tcross.getcol('UVW')

Successful readonly open of default-locked table /fastpool/data/20210226M-2GHz-1chan-9000int.ms: 25 columns, 1257676800 rows
CPU times: user 4min 35s, sys: 1min 17s, total: 5min 52s
Wall time: 6min 9s


In [4]:
%%time
with table('/fastpool/data/20210226M-2GHz-1chan-600int.ms') as t:
    vis = t.getcol('DATA')
    ant1 = t.getcol('ANTENNA1')
    ant2 = t.getcol('ANTENNA2')
    uvw = t.getcol('UVW')
vis[ant1 == ant2] = 0

Successful readonly open of default-locked table /fastpool/data/20210226M-2GHz-1chan-600int.ms: 25 columns, 1257676800 rows
CPU times: user 2min 59s, sys: 58.2 s, total: 3min 57s
Wall time: 3min 56s


In [6]:
print(vis.shape)
print(ant1.shape)
print(uvw.shape)

(1257676800, 1, 4)
(1257676800,)
(1257676800, 3)


In [13]:
nr_stations = len(t_ant)
nr_baselines = (nr_stations * (nr_stations - 1)) // 2
nr_channels = 1
nr_timesteps = 600
# Number of aterms to generate
nr_timeslots = 1
nr_correlations = 4
grid_size = 4096
image_size = 4096
subgrid_size = 32
kernel_size = 16
cell_size = image_size / grid_size

In [11]:
nr_stations

2048

In [12]:
nr_baselines

2096128

In [9]:
(nr_baselines) * 600 - uvw.shape[0]

0

I guess for these simulator-simulated measurement sets the autocorrelations just aren't even there...

In [15]:
aterms = util.get_identity_aterms(nr_timeslots, nr_stations, subgrid_size, nr_correlations)
aterms_offsets = util.get_example_aterms_offset(nr_timeslots, nr_timesteps)

In [16]:
aterms.shape

(1, 2048, 32, 32, 4)

In [17]:
aterms_offsets.shape

(2,)

In [17]:
aterms_offsets

array([0, 1], dtype=int32)

In [18]:
aterms[0,0,:,:,0]

array([[1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       ...,
       [1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j]],
      dtype=complex64)

Shapes of things from the demo script:
```
# Reshape data
antenna1_block = np.reshape(antenna1_block,
                            newshape=(nr_timesteps, nr_baselines))
antenna2_block = np.reshape(antenna2_block,
                            newshape=(nr_timesteps, nr_baselines))
uvw_block = np.reshape(uvw_block,
                       newshape=(nr_timesteps, nr_baselines, 3))
vis_block = np.reshape(vis_block,
                       newshape=(nr_timesteps, nr_baselines,
                                 nr_channels, nr_correlations))

# Transpose data
for t in range(nr_timesteps):
    for bl in range(nr_baselines):
        # Set baselines
        antenna1 = antenna1_block[t][bl]
        antenna2 = antenna2_block[t][bl]

        baselines[bl] = (antenna1, antenna2)

        # Set uvw
        uvw_ = uvw_block[t][bl]
        uvw[bl][t] = uvw_

        # Set visibilities
        visibilities[bl][t] = vis_block[t][bl]
```

# Now redo the input

In [3]:
with table('/fastpool/data/20210226M-2GHz-1chan-600int.ms') as t:
    vis = t.getcol('DATA')
    ant1 = t.getcol('ANTENNA1')
    ant2 = t.getcol('ANTENNA2')
    uvw = t.getcol('UVW')
    t_ant = table(t.getkeyword("ANTENNA"))
    t_spw = table(t.getkeyword("SPECTRAL_WINDOW"))
    frequencies = np.asarray(t_spw[0]['CHAN_FREQ'], dtype=np.float32)

vis[ant1 == ant2] = 0

nr_stations = len(t_ant)
nr_baselines = (nr_stations * (nr_stations - 1)) // 2
nr_channels = 1
nr_timesteps = 600
# Number of aterms to generate
nr_timeslots = 1
nr_correlations = 4
grid_size = 4096
image_size = 4096
subgrid_size = 32
kernel_size = 16
cell_size = image_size / grid_size

ant1 = ant1.reshape((nr_timesteps, nr_baselines))
ant2 = ant2.reshape((nr_timesteps, nr_baselines))
uvw_orig = uvw.reshape((nr_timesteps, nr_baselines, 3))
vis = vis.reshape((nr_timesteps, nr_baselines, nr_channels, nr_correlations))


vis = np.ascontiguousarray(np.swapaxes(vis, 0, 1).astype(idg.visibilitiestype))

Successful readonly open of default-locked table /fastpool/data/20210226M-2GHz-1chan-600int.ms: 25 columns, 1257676800 rows
Successful readonly open of default-locked table /fastpool/data/20210226M-2GHz-1chan-600int.ms/ANTENNA: 8 columns, 2048 rows
Successful readonly open of default-locked table /fastpool/data/20210226M-2GHz-1chan-600int.ms/SPECTRAL_WINDOW: 14 columns, 1 rows


In [4]:
baselines = np.zeros(shape=(nr_baselines), dtype=idg.baselinetype)
baselines['station1'] = ant1[0]
baselines['station2'] = ant2[0]

In [5]:
uvw = np.zeros(shape=(nr_baselines, nr_timesteps), dtype=idg.uvwtype)
uvw_orig = np.swapaxes(uvw_orig,0, 1)
uvw['u'] = uvw_orig[:, :, 0 ]
uvw['v'] = uvw_orig[:, :, 1]
uvw['w'] = uvw_orig[:, :, 2]

In [9]:
shift = np.zeros(3, dtype=np.float32)
w_step = 0.0

In [5]:
idg.baselinetype

dtype([('station1', '<i4'), ('station2', '<i4')])

In [8]:
proxy = idg.HybridCUDA.GenericOptimized()

In [ ]:
proxy.init_cache(subgrid_size, cell_size, w_step, shift)

In [7]:
grid = util.get_example_grid(nr_correlations, grid_size)

In [8]:
grid.shape

(4, 4096, 4096)

In [9]:
aterms = util.get_identity_aterms(nr_timeslots, nr_stations, subgrid_size, nr_correlations)
aterms_offsets = util.get_example_aterms_offset(nr_timeslots, nr_timesteps)

In [10]:
aterms.shape

(1, 2048, 32, 32, 4)

In [11]:
spheroidal = util.get_example_spheroidal(subgrid_size)

In [ ]:
%timeit
proxy.gridding(kernel_size, frequencies, vis, uvw, baselines, aterms, aterms_offsets, spheroidal)